In [2]:
# 解析 patent_md    

import re
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Union

CHS_NUM = {"一":1,"二":2,"三":3,"四":4,"五":5,"六":6,"七":7,"八":8,"九":9,"十":10}

def _chs_num_to_int(s: str) -> Optional[int]:
    s = s.strip()
    if s.isdigit():
        return int(s)
    if s == "十":
        return 10
    if len(s) == 2 and s[0] == "十" and s[1] in CHS_NUM:
        return 10 + CHS_NUM[s[1]]
    if len(s) == 2 and s[0] in CHS_NUM and s[1] == "十":
        return CHS_NUM[s[0]] * 10
    if len(s) == 3 and s[0] in CHS_NUM and s[1] == "十" and s[2] in CHS_NUM:
        return CHS_NUM[s[0]] * 10 + CHS_NUM[s[2]]
    if s in CHS_NUM:
        return CHS_NUM[s]
    return None

In [4]:
from collections import OrderedDict
import re
from pathlib import Path
from typing import List, Dict, Optional, Tuple, Union
from pypdf import PdfReader
from langchain_core.documents import Document


class PatentMarkdownParser:
    """
    输入：一个专利目录（包含 full.md 与 images/）
    输出：结构化信息字典：
      {
        'pubno': 'CN207225508U',
        'patent_name': '一种机器人足端结构',
        'applier': '杭州宇树科技有限公司地址xxxx……',
        'inventor': '王兴兴 杨知雨',
        'apply_time': '2017.10.16',
        'root_dir': 'D:\\path\\to\\专利目录',
        'fig_list': {
           '摘要图': ['摘要图', 'D:\\path\\to\\摘要图']
           '图1': ['本实用新型整体结构示意图', 'D:\\path\\to\\专利目录\\images\\xxx.jpg'],
           '图2': ['剖视图', 'D:\\...\\images\\yyy.jpg'],
           ...
        }
      }   
    """
    def __init__(self, patent_dir: Union[str, Path], md_name: str = "full.md", images_dir: str = "images"):
        self.root_dir = Path(patent_dir).resolve()
        self.md_path  = self.root_dir / md_name
        self.img_dir  = self.root_dir / images_dir
        if not self.md_path.exists():
            raise FileNotFoundError(f"找不到 Markdown：{self.md_path}")
        self.text = self.md_path.read_text(encoding="utf-8", errors="ignore")
        self.text = self.text.replace("\u3000", " ").replace("\ufeff", "")


    # --------------------------- 顶层 API ---------------------------
    def parse(self) -> Dict:
        title = self._extract_title()
        applier, addres = self._extract_applicant_and_address()

        info = {
            "pubno": self._extract_pubno(),
            "title": title,
            "applier": applier,
            "addres": addres,           # 按你的键名
            "inventor": self._extract_inventor(),
            "apply_time": self._extract_apply_date(),
            "root_dir": str(self.root_dir),
            "fig_list": self._build_fig_map()
        }
        return info

    # --------------------------- 字段抽取 ---------------------------
    def _extract_pubno(self) -> Optional[str]:
        for key in ["授权公告号", "公开号", "公告号"]:
            m = re.search(rf"{key}\s*([A-Z]{2}\d+[A-Z]?)", self.text)
            if m:
                return m.group(1).strip()
        m2 = re.search(r"(CN\d{6,}[A-Z]?)", self.text)
        return m2.group(1) if m2 else None

    def _extract_title(self) -> Optional[str]:
        # 常见：(54) 实用新型名称 / (54) 发明名称
        pat = r"(?:\(#?\s*54\)|\(54\)|#\s*\(54\))?[\s#]*[发明|实用新型]*名称\s*\n?([^\n#]+)"
        m = re.search(pat, self.text)
        if m:
            return m.group(1).strip()
        m2 = re.search(r"\(54\).*?名称\s*\n+([^\n#]+)", self.text)
        if m2:
            return m2.group(1).strip()
        m3 = re.search(r"#\s*\(54\).*?名称.*?\n+([^\n#]+)", self.text)
        if m3:
            return m3.group(1).strip()
        return None

    def _extract_applicant_and_address(self) -> Tuple[Optional[str], Optional[str]]:
        """
        仅返回公司名（applier）与地址（addres）。
        规则：
          - 在“专利权人 ...”或“申请人 ...”行里切分“地址[:：]?”
          - 若该行无地址，尝试在其后 200 个字符内再找一次“地址…”
        """
        m = re.search(r"(专利权人|申请人)\s*([^\n]+)", self.text)
        if not m:
            return None, None

        tail = m.group(2).strip()
        # 先在同一行切分
        m_line = re.search(r"^(.*?)\s*(?:地址[:：]?\s*)(.+)$", tail)
        if m_line:
            name = m_line.group(1).strip(" ，,;:；：")
            addr = m_line.group(2).strip()
            return (name or None), (addr or None)

        # 否则在“申请人/专利权人”后 200 字符里继续找“地址…”
        after = self.text[m.end(): m.end() + 200]
        m_near = re.search(r"地址[:：]?\s*([^\n]+)", after)
        if m_near:
            name = tail.strip(" ，,;:；：")
            addr = m_near.group(1).strip()
            return (name or None), (addr or None)

        # 实在没有地址，就只返回公司名整段
        name = tail.strip(" ，,;:；：")
        return (name or None), None

    def _extract_inventor(self) -> Optional[str]:
        m = re.search(r"(发明人)\s*([^\n]+)", self.text)
        if m:
            return m.group(2).strip()
        m2 = re.search(r"发明人[：:]\s*([^\n]+)", self.text)
        return m2.group(1).strip() if m2 else None

    def _extract_apply_date(self) -> Optional[str]:
        m = re.search(r"申请日\s*(\d{4}[.\-]\d{2}[.\-]\d{2})", self.text)
        return m.group(1) if m else None

    # --------------------------- 附图 & 图片 ---------------------------
    def _parse_figure_descriptions(self) -> Dict[str, str]:
        """
        解析 '# 附图说明' 段，形成 { '图1': 'xxx', '图2': 'yyy', ... }
        兼容：'图1为……；图2……；' 以及 '图1：……。图2：……。'
        """
        desc_map: Dict[str, str] = {}

        # 找到“附图说明”到下一个大标题之间的文本
        m = re.search(r"(#\s*附图说明[\s\S]*?)(?:\n#\s|\Z)", self.text)
        if not m:
            return desc_map
        block = m.group(1)

        # 把全角冒号/顿号等替换一下，减少花样
        norm = block.replace("：", ":").replace("；", ";").replace("。", "。")
        # 拆成句子（按；。分）
        parts = re.split(r"[;；。]\s*", norm)
        for p in parts:
            p = p.strip()
            if not p:
                continue
            # 匹配 “图X 为/是/： 描述 …”
            mm = re.match(r"^图\s*([0-9一二三四五六七八九十]+)\s*[:为是]\s*(.+)$", p)
            if not mm:
                # 也有“图X：描述”
                mm = re.match(r"^图\s*([0-9一二三四五六七八九十]+)\s*:\s*(.+)$", p)
            if mm:
                idx_raw = mm.group(1)
                idx = _chs_num_to_int(idx_raw) or (int(idx_raw) if idx_raw.isdigit() else None)
                if idx is None:
                    continue
                key = f"图{idx}"
                desc = mm.group(2).strip()
                desc_map[key] = desc
        return desc_map

    def _parse_images_in_order(self) -> List[Tuple[str, Optional[str]]]:
        """
        顺序解析 Markdown 中的图片：返回 [(abs_path, caption_line或None), ...]
        约定：图片行形如 ![](images/xxx.jpg)，下一行若以 '图X' 开头则当作图片标题行。
        """
        lines = self.text.splitlines()
        out: List[Tuple[str, Optional[str]]] = []
        for i, ln in enumerate(lines):
            m = re.search(r"!\[[^\]]*\]\(([^)]+)\)", ln)
            if not m:
                continue
            rel = m.group(1).strip()
            # 只处理 images/ 下的图
            if not rel.lower().startswith("images/"):
                continue
            abs_path = (self.root_dir / rel).resolve().as_posix()

            # 尝试用下一行/本行后缀作为 caption
            caption = None
            # 情况A：下一行单独 '图1' 或 '图1 说明……'
            if i + 1 < len(lines) and lines[i+1].strip().startswith("图"):
                caption = lines[i+1].strip()
            # 情况B：图片行末尾就带 '图1'（不常见）
            tail = ln.split(")", 1)[-1]
            if not caption and "图" in tail:
                tail_s = tail.strip()
                if tail_s.startswith("图"):
                    caption = tail_s
            out.append((abs_path, caption))
        return out

    def _build_fig_map(self) -> Dict[str, List[str]]:
        """
        综合“附图说明”的描述与图片顺序，生成 {图号: [描述, 绝对路径]}。
        对齐策略：
          1) 先按出现顺序给图片编号：第1张 → 图1，第2张 → 图2 …
          2) 若图片行紧随的 caption 行是 '图N …'，以该 N 为准覆盖顺序编号
          3) 再尝试用“附图说明”中的描述补全/覆盖文字
        """
        desc_map = self._parse_figure_descriptions()
        img_list = self._parse_images_in_order()

        fig_map: Dict[str, List[str]] = {}

        # 先顺序编号
        for idx, (abs_path, caption) in enumerate(img_list, start=1):
            key = f"图{idx}"

            # 如果 caption 行里写了具体编号（例如“图6 …”），按真实编号覆盖 key
            if caption:
                mm = re.match(r"^图\s*([0-9一二三四五六七八九十]+)\s*(.*)", caption)
                if mm:
                    # 解析 “图X + 其余描述”
                    idx_raw, rest = mm.group(1), mm.group(2).strip()
                    real_idx = _chs_num_to_int(idx_raw) or (int(idx_raw) if idx_raw.isdigit() else None)
                    if real_idx:
                        key = f"图{real_idx}"
                        if rest:  # “图X 副标题”
                            fig_map[key] = [rest, abs_path]
                            continue

            # 没有明确描述时，先放空描述
            fig_map.setdefault(key, ["", abs_path])

        # 用“附图说明”的描述覆盖空描述
        for k, v in fig_map.items():
            if not v[0]:  # 描述为空
                if k in desc_map:
                    v[0] = desc_map[k]

        return fig_map


# =============== 批量处理工具（可选） ===============
def parse_patent_dir(patent_dir: Union[str, Path]) -> Dict:
    """解析单个专利目录，返回结构化字典"""
    return PatentMarkdownParser(patent_dir).parse()

def parse_patent_tree(root: Union[str, Path]) -> List[Dict]:
    """
    扫描根目录，凡是包含 full.md 的子目录都解析。
    返回每个专利的结构化结果列表。
    """
    root = Path(root)
    results = []
    for md in root.rglob("full.md"):
        try:
            results.append(PatentMarkdownParser(md.parent).parse())
        except Exception as e:
            print(f"[WARN] 解析失败：{md.parent} -> {e}")
    return results




In [7]:
if __name__ == "__main__":
    import os 
    # 示例：传入单个专利目录
    sample_dir = r"D:\ddesktop\agentdemos\codespace\zhuanliParser\result"  # 里面有 full.md 和 images/
    for dir in os.listdir(sample_dir):
        data = parse_patent_dir(os.path.join(sample_dir, dir))
        from pprint import pprint
        pprint(data)

{'addres': '310051浙江省杭州市滨江区聚业路26号金绣国际科技中心B座106室',
 'applier': '杭州宇树科技有限公司',
 'apply_time': '2017.10.16',
 'fig_list': {'图1': ['',
                     'D:/ddesktop/agentdemos/codespace/zhuanliParser/result/CN201721328994.5-一种机器人足端结构.pdf-1bd0a73e-8078-4700-ac04-c2220af5a453/images/88fe5b0e5b16afc42f78352f8df13f234eeb58b135cdcfdfea80134da7580363.jpg'],
              '图2': ['',
                     'D:/ddesktop/agentdemos/codespace/zhuanliParser/result/CN201721328994.5-一种机器人足端结构.pdf-1bd0a73e-8078-4700-ac04-c2220af5a453/images/f6f4294118dd28c8f795d7c633a1e3bd99fad6f9422678f4dc7439d64982e4a8.jpg'],
              '图3': ['',
                     'D:/ddesktop/agentdemos/codespace/zhuanliParser/result/CN201721328994.5-一种机器人足端结构.pdf-1bd0a73e-8078-4700-ac04-c2220af5a453/images/2649e5073d9e380fe64058ca443c4bcb2bf8d6c0183067d83a025b530a406f7e.jpg'],
              '图4': ['',
                     'D:/ddesktop/agentdemos/codespace/zhuanliParser/result/CN201721328994.5-一种机器人足端结构.pdf-1bd0a73e-8078-4700-

In [10]:
from collections import OrderedDict
import os, re
from pathlib import Path
from pypdf import PdfReader
from langchain_core.documents import Document
from typing import List, Dict, Optional, Tuple 

class zhuanli_parser:
    """
    专利文档解析器类
    
    功能：
    1. 从 Markdown 格式的专利文档中提取元数据
    2. 解析专利的基本信息（申请公布号、专利名称、申请人、发明人、申请时间）
    3. 提取并关联附图信息（图片路径、描述）
    4. 清理文档内容，生成 LangChain Document 对象
    
    主要处理流程：
    - 解析专利基本信息
    - 提取摘要图和附图说明
    - 建立图号与图片路径的映射关系
    - 过滤掉原始 Markdown 中的图片标记
    - 返回包含完整元数据的 Document 对象
    """
    
    def __init__(self, markdown_file: str):
        """
        初始化专利解析器
        
        Args:
            markdown_file: 专利 Markdown 文件的路径
        """
        self.markdown_file = markdown_file
        self.ims_dir = Path(markdown_file).parent / "images"
        
        # 定义元数据结构模板
        self.meta_schema = OrderedDict({
            "pubno": str,           # 申请公布号
            "patent_name": str,     # 专利名称
            "applier": str,         # 申请人
            "inventor": str,        # 发明人
            "apply_time": str,      # 申请时间
            "root_dir": str(Path(markdown_file).parent.parent.absolute())+".pdf",  # 根目录路径
            "fig_list": dict,       # 图片列表：{"图1": ["描述", "绝对路径"]}
        })
    
    def __call__(self):
        """使类实例可调用，直接执行解析流程"""
        return self.pipeline()  
    
    def _load_md_text(self) -> str:
        """加载 Markdown 文件内容"""
        md_text = Path(self.markdown_file).read_text(encoding='utf-8')
        return md_text
    
    @staticmethod
    def _chs_num_to_int(s: str) -> Optional[int]:
        """
        中文数字转阿拉伯数字的工具函数
        
        支持格式：
        - 单字：一、二、三...九、十
        - 十开头：十、十一、十二...
        - 两位数：二十、二十三...
        - 纯阿拉伯数字：直接转换
        
        Args:
            s: 中文数字字符串
            
        Returns:
            转换后的整数，转换失败返回 None
        """
        m = {"零":0,"一":1,"二":2,"两":2,"三":3,"四":4,"五":5,"六":6,"七":7,"八":8,"九":9,"十":10}
        s = s.strip()
        if not s: 
            return None
            
        # 仅 "十" 开头（十、十一、二十、二十三）
        if s == "十": 
            return 10
        if len(s) == 1 and s in m: 
            return m[s]
            
        # 十X 格式
        if s[0] == "十":
            tail = m.get(s[1:], 0) if s[1:] else 0
            return 10 + tail
            
        # X十 或 X十一 格式
        if "十" in s:
            left, right = s.split("十", 1)
            left_v = m.get(left, 1) if left else 1
            right_v = m.get(right, 0) if right else 0
            return left_v * 10 + right_v
            
        # 纯数字
        if s.isdigit(): 
            return int(s)
            
        # 单字数字
        if s in m: 
            return m[s]
            
        return None
    
    @staticmethod
    def _find_section(text: str, title_pattern: str) -> Optional[str]:
        r"""
        截取某个标题段落的内容（从标题到下一个标题或文件末尾）
        
        Args:
            text: 文档全文
            title_pattern: 标题匹配模式（不带 # 的正则表达式）
                          例如：r'附图说明' 或 r'\(57\)\s*摘要'
            
        Returns:
            标题段落的内容，未找到返回 None
        """
        hdr = re.search(rf"^\s*#{{1,3}}\s*{title_pattern}\s*$", text, flags=re.MULTILINE)
        if not hdr:
            return None
        start = hdr.end()
        nxt = re.search(r"^\s*#\s+", text[start:], flags=re.MULTILINE)
        return text[start: start + nxt.start()] if nxt else text[start:]
    
    def _extract_abstract_img(self) -> Tuple[str, Dict[str, List[str]]]:
        """
        提取摘要图信息
        
        从 "(57)摘要" 段落中提取第一张图片（通常摘要只放一张）
        
        Returns:
            tuple: (清理后的摘要文本, {"fig_list": {"abs_im": ["摘要图", 绝对路径]}})
                  若无摘要图，abs_im 为 None
        """
        # 查找 "(57)摘要" 段落
        block = self._find_section(self.text, r"\(?57\)?\s*摘要")
        if not block:
            return "", {"fig_list": {"abs_im": None}}

        # 提取第一张图片
        img_m = re.search(r'!\[.*?\]\((.*?)\)', block)
        if not img_m:
            return block.strip(), {"fig_list": {"abs_im": None}}

        rel_path = img_m.group(1).strip()
        abs_path = str((Path(self.markdown_file).parent / rel_path).resolve())

        # 清理图片标记，保留纯文本
        cleaned = re.sub(r'!\[.*?\]\(.*?\)\s*\n?', '', block).strip()

        return cleaned, {"fig_list": {"abs_im": ["摘要图", abs_path]}}

    def _parse_figure_descriptions(self) -> Dict[str, str]:
        """
        解析附图说明段落，提取图号到描述的映射
        
        支持的格式：
        - "[0017] 图1为……；[0018] 图2：……。"
        - "图一是……；图二为……。"
        - "图1: ……" / "图1：……"
        
        Returns:
            图号到描述的映射字典：{'图1': '描述文本', '图2': '描述文本', ...}
        """
        desc_map: Dict[str, str] = {}

        block = self._find_section(self.text, r"附图说明")
        if not block:
            return desc_map

        # 统一中英文标点符号
        block = block.replace("：", ":").replace("；", ";")

        # 全局查找图号和描述的匹配模式
        pat = re.compile(
            r"(?:\[\d+\]\s*)?"                # 可选的前缀编号如 [0017]
            r"图\s*([0-9一二三四五六七八九十]+)\s*"  # 图号（支持中文数字）
            r"(?:为|是|:)\s*"                   # 连接词
            r"(.+?)"                          # 非贪婪匹配描述内容
            r"(?=(?:；|;|。|\.|\n|$))",        # 直到句末标点、换行或文末
            flags=re.IGNORECASE | re.DOTALL
        )

        for num, desc in pat.findall(block):
            # 转换图号为标准格式
            idx = self._chs_num_to_int(num) or (int(num) if num.isdigit() else None)
            if not idx:
                continue
            key = f"图{idx}"
            # 清理描述文本结尾的标点和空白
            desc_map[key] = re.sub(r"[\s;；。.\u3000]+$", "", desc.strip())

        return desc_map

    def _build_fig_map(self) -> Dict[str, str]:
        """
        从全文扫描图片，建立图号到图片路径的映射
        
        兼容情况：
        - 图片与"图X"标签在同一行
        - 图片与"图X"标签在下一行
        - 通过两个空格实现的 Markdown 软换行
        
        Returns:
            图号到绝对路径的映射：{'图1': '绝对路径', ...}
        """
        img_map: Dict[str, str] = {}
        base_dir = Path(self.markdown_file).parent

        # 查找图片和对应的图号标签
        # 支持图片后 0-2 行内出现图号
        img_iter = re.finditer(
            r'!\[.*?\]\((.*?)\)\s*(?:\n[ \t]*){0,2}(图[0-9一二三四五六七八九十]+)',
            self.text, flags=re.IGNORECASE
        )
        
        for m in img_iter:
            rel_path = m.group(1).strip()
            tag = m.group(2).strip()          # 如 "图1" / "图一"
            
            # 规范化图号
            mnum = re.match(r"图\s*([0-9一二三四五六七八九十]+)", tag)
            if not mnum:
                continue
            idx_raw = mnum.group(1)
            idx = self._chs_num_to_int(idx_raw) or (int(idx_raw) if idx_raw.isdigit() else None)
            if not idx:
                continue
            key = f"图{idx}"

            # 构建绝对路径并验证文件存在
            abs_path = str((base_dir / rel_path).resolve())
            if Path(abs_path).exists():
                img_map[key] = abs_path

        return img_map

    def _extract_img_metadata(self) -> Dict[str, Dict[str, List[str]]]:
        """
        整合图片元数据：摘要图 + 附图说明 + 图片路径
        
        Returns:
            完整的图片元数据字典：
            {"fig_list": {
                "abs_im": ["摘要图", <绝对路径>] 或 None,
                "图1": ["描述或空字符串", <绝对路径>] ...
            }}
        """
        # 获取摘要图信息
        _, abs_meta = self._extract_abstract_img()
        abs_item = abs_meta["fig_list"].get("abs_im", None)

        # 获取附图说明中的描述
        desc_map = self._parse_figure_descriptions()

        # 获取全文图片路径映射
        img_map = self._build_fig_map()

        # 整合信息：以有路径的图为准，描述来自desc_map，没有则为空串
        fig_list: Dict[str, List[str]] = {}
        for k, path in img_map.items():
            desc = desc_map.get(k, "")
            fig_list[k] = [desc, path]

        # 添加摘要图（如果存在）
        if abs_item and isinstance(abs_item, list) and len(abs_item) == 2:
            fig_list["abs_im"] = abs_item

        return {"fig_list": fig_list}
    
    def _extract_imMatadata(self, content: str) -> dict:
        """
        从 Markdown 文本中提取图片元数据（旧版本方法，保留用于兼容）
        
        处理逻辑：
        1. 查找"附图说明"章节
        2. 建立图号到相对路径的映射
        3. 逐行提取图号和描述的关联
        
        Args:
            content: Markdown 文档内容
            
        Returns:
            图片元数据字典：{"fig_list": {"图1":[描述,绝对路径], ...}}
            若无附图章节，返回空字典
        """
        # 1. 提取"附图说明"标题及其内容（直到下一个标题或文件结尾）
        pattern = re.compile(
            r'^(#{1,3})\s*附图说明\s*\n+\s*([\s\S]*?)(?=^#{1,3}|\Z)',
            re.MULTILINE)
        match = pattern.search(content)
        if not match:
            print(f"⚠️ markdown file {os.path.basename(self.markdown_file)} 未找到 '附图说明' 章节，跳过处理。")
            return {"fig_list": {}}

        body = match.group(0)
        
        # 2. 建立图号到相对路径的映射（全局搜索）
        img_map: Dict[str, str] = {
            f"图{num}": path.strip()
            for path, num in re.findall(
                r'!\[.*?\]\((.*?)\)\s*\n?\s*图(\d+)', content, flags=re.I
            )
        }
            
        # 3. 在附图说明段落内逐行提取图号和描述
        fig_dict: Dict[str, list] = {}
        for line in body.splitlines():
            # 支持格式：[0050] 图1是...； 或 图1是...；
            m = re.search(r'图(\d+)是(.*?)[，；:：,.;。]', line.strip())
            if not m:
                continue
            num, desc = m.groups()
            key = f"图{num}"
            rel_path = img_map.get(key)
            
            if rel_path:
                abs_path = str((Path(self.markdown_file).parent / rel_path).resolve())
                if Path(abs_path).exists():
                    fig_dict[key] = [desc.strip(), abs_path]
                else:
                    print(f"⚠️ 图片不存在：{abs_path}")
                    
        return {"fig_list": fig_dict}
       
    def _extract_abstract_im(self, markdown_text: str) -> tuple[str, list]:
        """
        提取摘要图信息（旧版本方法，保留用于兼容）
        
        Args:
            markdown_text: Markdown 文档内容
            
        Returns:
            tuple: (清理后的摘要文本, 摘要图元数据)
        """
        # 1. 查找(57)摘要整个块
        m = re.search(
            r'^(#{1,3})\s*\(57\)摘要\s*\n(.*?)(?=^#{1,3}|\Z)',
            markdown_text, flags=re.M | re.S
        )
        if not m:
            return "", {"fig_list": {"abs_im": None}}

        abstract_block = m.group(2)

        # 2. 提取图片路径
        img_m = re.search(r'!\[.*?\]\((.*?)\)', abstract_block)
        if not img_m:
            return abstract_block.strip(), {"fig_list": {"abs_im": None}}

        rel_path = img_m.group(1).strip()
        abs_path = str((Path(self.markdown_file).parent / rel_path).resolve())

        # 3. 清理图片标记（整行删除）
        cleaned = re.sub(r'!\[.*?\]\(.*?\)\s*\n?', '', abstract_block).strip()

        return cleaned, {"fig_list": {"abs_im": ["摘要配图", abs_path]}}
        
    def _extract_meta_blocks(self, text: str) -> dict[str, str]:
        """
        从专利文本中提取基本元数据信息
        
        提取字段：
        - 专利名称（从 "(54)实用新型名称" 段落）
        - 申请时间（从 "(22)申请日" 字段）
        - 申请人（从 "(73)专利权人" 字段，只保留名称）
        - 发明人（从 "(72)发明人" 字段）
        
        Args:
            text: 专利文档文本
            
        Returns:
            包含基本信息的字典
        """
        # 1. 提取实用新型名称
        name = re.search(r'(?m)^#\s*\(54\)\s*实用新型名称\s*\n(.+)', text)
        patent_name = name.group(1).strip() if name else ""

        # 2. 提取其余字段（申请日、专利权人、发明人）
        m = re.search(
            r'\(22\)申请日\s*([^\s\n]+).*?'      # 申请日
            r'\(73\)专利权人\s*([^\s\n]+).*?'    # 申请人（只保留名称，忽略地址）
            r'\(72\)发明人\s*([^\s\n]+)',        # 发明人
            text, flags=re.S
        )
        if not m:
            return {"patent_name": patent_name,
                    "apply_time": "", "applier": "", "inventor": ""}

        apply_time, applier, inventor = m.groups()
        return {
            "patent_name": patent_name,
            "apply_time": apply_time.strip(),
            "applier": applier.strip(),
            "inventor": inventor.strip()
        }
    
    def _extract_pubno(self) -> dict[str, str]:
        """
        从对应的 PDF 文件中提取申请公布号
        
        从专利 PDF 第一页的最后一行提取申请公布号（格式如：CN202021894937U）
        
        Returns:
            包含申请公布号的字典：{"pubno": "CN..."}
            
        Raises:
            ValueError: 未找到申请公布号时抛出异常
        """
        # 构造对应的 PDF 文件路径
        # pdfp = str(self.markdown_file)[:-3] + "_origin.pdf"
        pdfp = Path(self.markdown_file).parent.rglob("*_origin.pdf")
        reader = PdfReader(pdfp)
        text_1 = reader.pages[0].extract_text() or ""
        last_line = text_1.strip().splitlines()[-1]

        # 去掉空格后匹配申请公布号格式
        compact = re.sub(r'\s+', '', last_line.upper())
        m = re.search(r'(CN[A-Z0-9]{9,13})', compact)
        if m:
            pubno_str = m.group(0)
            return {"pubno": pubno_str}
        else:
            raise ValueError(f"专利 {os.path.basename(pdfp)} 未找到申请公布号的字符串")    
        
    def _filter_endsl(self, langchain_md_text: str) -> str:
        """
        删除文档末尾所有连续的"图片 + 图X"块
        
        Args:
            langchain_md_text: Markdown 文档内容
            
        Returns:
            清理后的文档内容
        """
        new_text = re.sub(
            r'(\n\s*!\[.*?\]\([^)]+\)\s*\n\s*图\d+\s*)+$',
            '',
            langchain_md_text,
            flags=re.MULTILINE | re.IGNORECASE
        ).rstrip() + '\n'
        return new_text
    
    def _filter_lines(self, langchain_md_text: str) -> str:
        """
        删除文档中所有的图片标记和图号标签
        
        清理所有形如以下格式的内容块：
            ![...](...)
            图X...
        （支持中间有空行的情况）
        
        注意：摘要图会被保留在摘要段落中单独处理
        
        Args:
            langchain_md_text: 原始 Markdown 文档内容
            
        Returns:
            清理后的纯文本文档内容
        """
        # (?s) 标志让 . 匹配换行符
        pattern = re.compile(
            r'!\[.*?\]\([^)]*\)[ \t]*(?:\n[ \t]*)*\n?[ \t]*图\d+[：:\s]*[^\n]*(?:\n|$)',
            flags=re.IGNORECASE | re.MULTILINE
        )
        return pattern.sub('', langchain_md_text)

    def pipeline(self) -> Document:
        """
        主处理流程：执行完整的专利文档解析流程
        
        处理步骤：
        1. 加载 Markdown 文档
        2. 提取图片元数据（图号、描述、路径）
        3. 过滤掉原文档中的图片标记
        4. 提取基本元数据（申请公布号、专利信息）
        5. 整合所有元数据
        6. 创建并返回 LangChain Document 对象
        
        Returns:
            包含清理后内容和完整元数据的 LangChain Document 对象
        """
        # 加载 Markdown 文档内容
        md_text = self._load_md_text()
        
        # 获取图片元数据
        fig_lists = self._extract_imMatadata(md_text) 
        
        # 过滤掉原 Markdown 中嵌入的图片标记
        md_text_filtered = self._filter_lines(md_text)
        
        # 创建 LangChain Document 对象
        loaded_docs = Document(page_content=md_text_filtered, metadata={})
        
        # 获取申请公布号元数据
        pubno = self._extract_pubno()
        
        # 获取专利基本信息元数据
        meta_blocks = self._extract_meta_blocks(md_text_filtered)
        
        # 更新元数据结构
        self.meta_schema.update(fig_lists)
        self.meta_schema.update(meta_blocks)
        self.meta_schema.update(pubno)
        
        # 同步更新到 Document 的 metadata
        loaded_docs.metadata.update(**self.meta_schema)
        
        return loaded_docs


if __name__ == '__main__':
    # 使用示例
    markdown_file = r"D:\ddesktop\agentdemos\codespace\zhuanliParser\result\CN202021894937.5-一种结构紧凑的回转动力单元以及应用其的机器人.pdf-c96c9eb4-261f-46aa-8b93-8211ff1d937d\full.md"
    
    
    # 创建解析器实例
    zhuanli = zhuanli_parser(markdown_file=markdown_file)
    
    # 执行解析
    loaded_docs = zhuanli()
    
    # 输出结果
    print(loaded_docs.metadata)

AttributeError: 'generator' object has no attribute 'seek'

In [ ]:
# RAG图文输出
#    核心逻辑：构建 文本段-图片 内联

# method1:   嵌入特定字符 $摘要图$ 到正文中。 很呆，污染了原始的文本
# method2：  段落级别的图片关联（推荐）
#                 保持原文档结构不变
#                 在 metadata 中增加段落到图片的映射关系
#                 智能体根据检索到的段落类型自动匹配图片
# method3:  智能段落识别法（最优）
#                利用语义匹配识别段落类型
#                自动关联相关图片
#                支持多种图片类型（摘要图、附图等）


In [ ]:
# method3  智能段落识别

class PatentMarkdownParser:
    """
    将专利解析后的 full.md 转为 LangChain Document，并抽取结构化元数据。
    目录结构（示例）：
        <patent_dir>/
          ├─ full.md
          ├─ xx_origin.pdf            # 原始专利PDF（推荐命名）
          └─ images/
              ├─ a.jpg
              ├─ b.jpg
              └─ ...

    解析要点：
      1) 基本元数据：title / apply_time / applier / address / inventor / pubno
      2) 图片元数据：摘要图（abs_im）+ 附图（图1/图2/...）→ {"fig_list": {"图1": ["描述", "绝对路径"], ...}}
      3) 清理正文中的图片标记（![](...) + “图X ...”行）后作为向量化文本
    """

    # --------------------------- 初始化 ---------------------------
    def __init__(self, markdown_file: str):
        self.markdown_file = str(Path(markdown_file))
        self.base_dir: Path = Path(self.markdown_file).parent
        self.images_dir: Path = self.base_dir / "images"

        # 解析过程中的缓存文本（pipeline 内赋值）
        self.text: str = ""

        # 目标元数据骨架（有序，方便可视化/调试）
        self.meta_schema = OrderedDict({
            "pubno": "",           # 授权/公告号，如 CN20xxxx
            "title": "",           # 专利标题（(54) 实用新型/发明 名称）
            "applier": "",         # 专利权人（只保留公司名）
            "address": "",         # 地址字段单独保存
            "inventor": "",        # 发明人
            "apply_time": "",      # (22) 申请日
            "root_dir": str(self.base_dir.resolve()),  # 专利目录
            "pdf_path": "",        # 原始 PDF 的绝对路径（尽力猜测）
            "fig_list": {},        # {"abs_im": ["摘要图", abs_path], "图1": ["描述", abs_path], ...}
        })

    # --------------------------- 主入口 ---------------------------
    def __call__(self) -> Document:
        return self.pipeline()

    # --------------------------- 工具：加载全文 ---------------------------
    def _load_md_text(self) -> str:
        return Path(self.markdown_file).read_text(encoding="utf-8")

    # --------------------------- 工具：中文数字 → 阿拉伯数字 ---------------------------
    @staticmethod
    def _chs_num_to_int(s: str) -> Optional[int]:
        """
        仅处理 1~99 的常见中文数字（十、十一、二十、二十三…），够用即可。
        """
        m = {"零":0,"一":1,"二":2,"两":2,"三":3,"四":4,"五":5,"六":6,"七":7,"八":8,"九":9,"十":10}
        s = s.strip()
        if not s: return None
        if s == "十": return 10
        if len(s) == 1 and s in m: return m[s]
        # 十X
        if s[0] == "十":
            tail = m.get(s[1:], 0) if s[1:] else 0
            return 10 + tail
        # X十 / X十Y
        if "十" in s:
            left, right = s.split("十", 1)
            left_v = m.get(left, 1) if left else 1
            right_v = m.get(right, 0) if right else 0
            return left_v * 10 + right_v
        if s.isdigit(): return int(s)
        if s in m: return m[s]
        return None

    # --------------------------- 工具：截取 # 标题段 ---------------------------
    @staticmethod
    def _find_section(text: str, title_pattern: str) -> Optional[str]:
        r"""
        获取形如 "# 附图说明" / "# (57) 摘要" 段落（到下一个 # 或文末）。
        title_pattern：不含 # 的正则，如 r'附图说明' 或 r'\(57\)\s*摘要'
        """
        hdr = re.search(rf"^\s*#{{1,3}}\s*{title_pattern}\s*$", text, flags=re.MULTILINE)
        if not hdr:
            return None
        start = hdr.end()
        nxt = re.search(r"^\s*#\s+", text[start:], flags=re.MULTILINE)
        return text[start: start + nxt.start()] if nxt else text[start:]

    # --------------------------- 图片：摘要图 ---------------------------
    def _extract_abstract_img(self) -> Tuple[str, Dict[str, List[str]]]:
        """
        返回: (cleaned_abstract_text, {"fig_list": {"abs_im": ["摘要图", abs_path]}})
        若无摘要图，abs_im 为 None
        """
        block = self._find_section(self.text, r"\(?57\)?\s*摘要")
        if not block:
            return "", {"fig_list": {"abs_im": None}}

        img_m = re.search(r'!\[.*?\]\((.*?)\)', block)
        if not img_m:
            return block.strip(), {"fig_list": {"abs_im": None}}

        rel_path = img_m.group(1).strip()
        abs_path = str((self.base_dir / rel_path).resolve())
        cleaned = re.sub(r'!\[.*?\]\(.*?\)\s*\n?', "", block).strip()
        return cleaned, {"fig_list": {"abs_im": ["摘要图", abs_path]}}

    # --------------------------- 图片：附图说明（图号→描述） ---------------------------
    def _parse_figure_descriptions(self) -> Dict[str, str]:
        """
        返回 {'图1': 'xxx', '图2': 'yyy', ...}
        兼容：
          - "[0017] 图1为……；[0018] 图2：……。"
          - "图一是……；图二为……。"
          - "图1: ……" / "图1：……"
        """
        desc_map: Dict[str, str] = {}
        block = self._find_section(self.text, r"附图说明")
        if not block:
            return desc_map

        # 统一标点，便于匹配
        block = block.replace("：", ":").replace("；", ";")

        pat = re.compile(
            r"(?:\[\d+\]\s*)?"                # 可选编号 [0017]
            r"图\s*([0-9一二三四五六七八九十]+)\s*"
            r"(?:为|是|:)\s*"
            r"(.+?)"                          # 描述
            r"(?=(?:；|;|。|\.|\n|$))",        # 句末/换行/文末
            flags=re.IGNORECASE | re.DOTALL
        )

        for num, desc in pat.findall(block):
            idx = self._chs_num_to_int(num) or (int(num) if num.isdigit() else None)
            if not idx:
                continue
            key = f"图{idx}"
            desc_map[key] = re.sub(r"[\s;；。.\u3000]+$", "", desc.strip())

        return desc_map

    # --------------------------- 图片：全文图片（图号→路径） ---------------------------
    def _build_fig_map(self) -> Dict[str, str]:
        """
        全文扫描图片，尝试关联到后续最近的“图X”标签。
        兼容：图片与“图X”在同一行 / 下一行 / 连续两行软换行。
        返回 {'图1': abs_path, ...}
        """
        img_map: Dict[str, str] = {}

        img_iter = re.finditer(
            r'!\[.*?\]\((.*?)\)\s*(?:\n[ \t]*){0,2}(图[0-9一二三四五六七八九十]+)',
            self.text, flags=re.IGNORECASE
        )
        for m in img_iter:
            rel_path = m.group(1).strip()
            tag = m.group(2).strip()
            mnum = re.match(r"图\s*([0-9一二三四五六七八九十]+)", tag)
            if not mnum:
                continue
            idx_raw = mnum.group(1)
            idx = self._chs_num_to_int(idx_raw) or (int(idx_raw) if idx_raw.isdigit() else None)
            if not idx:
                continue
            key = f"图{idx}"
            abs_path = str((self.base_dir / rel_path).resolve())
            if Path(abs_path).exists():
                img_map[key] = abs_path

        return img_map

    # --------------------------- 图片：整合 摘要图 + 附图说明 + 路径 ---------------------------
    def _extract_img_metadata(self) -> Dict[str, Dict[str, List[str]]]:
        """
        返回:
          {"fig_list": {
              "abs_im": ["摘要图", <abs_path>] 或 None,
              "图1": ["描述或空字符串", <abs_path>], ...
          }}
        """
        _, abs_meta = self._extract_abstract_img()   # {"fig_list": {"abs_im": [...]} 或 None}
        abs_item = abs_meta["fig_list"].get("abs_im")

        desc_map = self._parse_figure_descriptions() # {'图1': '...', ...}
        img_map  = self._build_fig_map()             # {'图1': 'abs_path', ...}

        fig_list: Dict[str, List[str]] = {}
        for k, path in img_map.items():
            desc = desc_map.get(k, "")
            fig_list[k] = [desc, path]

        if abs_item and isinstance(abs_item, list) and len(abs_item) == 2:
            fig_list["abs_im"] = abs_item

        return {"fig_list": fig_list}

    # --------------------------- 文本：清理图片标记 ---------------------------
    def _filter_lines(self, md_text: str) -> str:
        """
        删除形如：
           ![...](...)
           图X...
        的成对块（支持中间空行）。摘要图在向量化阶段通常也不需要，可一并清除。
        """
        pattern = re.compile(
            r'!\[.*?\]\([^)]*\)[ \t]*(?:\n[ \t]*)*\n?[ \t]*图[0-9一二三四五六七八九十]+[：:\s]*[^\n]*(?:\n|$)',
            flags=re.IGNORECASE | re.MULTILINE
        )
        out = pattern.sub('', md_text)
        # 同时清理裸图片（摘要图等未伴随“图X”标注）
        out = re.sub(r'!\[.*?\]\([^)]*\)\s*\n?', '', out)
        return out.strip() + "\n"

    # --------------------------- 元数据：PDF 公告号提取 ---------------------------
    def _extract_pubno(self) -> Dict[str, str]:
        """
        从原始 PDF 首页末行提取公告号（CN...）。若目录下存在 *_origin.pdf 优先取之；
        否则尝试取当前目录下任一 .pdf；找不到则抛异常提示。
        """
        pdf_path = self._guess_pdf_path()
        if not pdf_path:
            raise FileNotFoundError(f"未找到原始 PDF 文件（建议命名 *_origin.pdf），目录：{self.base_dir}")

        reader = PdfReader(pdf_path)
        text_1 = reader.pages[0].extract_text() or ""
        last_line = (text_1.strip().splitlines() or [""])[-1]

        compact = re.sub(r'\s+', '', last_line.upper())
        m = re.search(r'(CN[A-Z0-9]{9,13})', compact)
        if m:
            return {"pubno": m.group(1), "pdf_path": str(Path(pdf_path).resolve())}
        else:
            # 个别版式不在页尾，则退化：全页扫描一次
            compact_all = re.sub(r'\s+', '', text_1.upper())
            m2 = re.search(r'(CN[A-Z0-9]{9,13})', compact_all)
            if m2:
                return {"pubno": m2.group(1), "pdf_path": str(Path(pdf_path).resolve())}
            raise ValueError(f"PDF 未找到公告号 CN**** ：{os.path.basename(pdf_path)}")

    def _guess_pdf_path(self) -> Optional[str]:
        """
        猜测原始 PDF 路径：
          1) 同目录 *_origin.pdf
          2) 同目录 *.pdf
        """
        cands = list(self.base_dir.glob("*_origin.pdf"))
        if not cands:
            cands = list(self.base_dir.glob("*.pdf"))
        return str(cands[0]) if cands else None

    # --------------------------- 元数据：(54)/(22)/(73)/(72) ---------------------------
    def _extract_meta_blocks(self, text: str) -> Dict[str, str]:
        """
        提取：
          - title: (54) 实用新型名称 / 发明名称
          - apply_time: (22) 申请日
          - applier, address: (73) 专利权人（名称、地址分离）
          - inventor: (72) 发明人
        针对 OCR/解析误差做了尽量鲁棒的正则。
        """
        # (54) 标题（“实用新型名称 / 发明名称”均兼容）
        m_title = re.search(r'(?m)^#\s*\(54\)\s*(?:实用新型|发明)名称\s*\n(.+)$', text)
        title = m_title.group(1).strip() if m_title else ""

        # (22) 申请日
        m_apply = re.search(r'\(22\)\s*申请日\s*([0-9.\-年月日/]+)', text)
        apply_time = (m_apply.group(1).strip() if m_apply else "").replace("年","." ).replace("月",".").replace("日","").strip(".")

        # (73) 专利权人：尽量分离“地址”
        # 典型："(73)专利权人 杭州宇树科技有限公司 地址 310051浙江省杭州市..."
        m_73 = re.search(r'\(73\)\s*专利权人\s*([^\n]+)', text)
        applier = ""
        address = ""
        if m_73:
            line = re.sub(r'\s+', '', m_73.group(1))
            # 优先按“地址”切分
            if "地址" in line:
                parts = line.split("地址", 1)
                applier = parts[0].strip()
                address = parts[1].strip()
            else:
                # 若无“地址”，退化：取到空白前的公司名（常见公司后缀）
                m_company = re.match(r'(.+?(?:公司|研究院|大学|学院|研究所|中心))', line)
                applier = m_company.group(1) if m_company else line

        # (72) 发明人
        m_72 = re.search(r'\(72\)\s*发明人\s*([^\n]+)', text)
        inventor = (m_72.group(1).strip() if m_72 else "")

        return {
            "title": title,
            "apply_time": apply_time,
            "applier": applier,
            "address": address,
            "inventor": inventor
        }

    # --------------------------- 文末图片块清理（可选） ---------------------------
    def _filter_trailing_image_blocks(self, text: str) -> str:
        """
        删除末尾连续出现的 “图片 + 图X” 块（若你的 full.md 末尾集中放图，可开启此步）。
        """
        new_text = re.sub(
            r'(\n\s*!\[.*?\]\([^)]+\)\s*\n\s*图[0-9一二三四五六七八九十]+\s*)+$',
            '',
            text,
            flags=re.MULTILINE | re.IGNORECASE
        ).rstrip() + '\n'
        return new_text

    # --------------------------- 总流水线 ---------------------------
    def pipeline(self) -> Document:
        # 1) 读取全文
        self.text = self._load_md_text()

        # 2) 图片元数据（摘要图 + 附图）
        img_meta = self._extract_img_metadata()        # {"fig_list": {...}}

        # 3) 清理正文图片标记，得到纯文本（供向量化）
        md_text_filtered = self._filter_lines(self.text)

        # 4) 结构化字段：title/apply_time/applier/address/inventor
        meta_blocks = self._extract_meta_blocks(md_text_filtered)

        # 5) PDF 公告号 & pdf_path
        pubno_info = self._extract_pubno()             # {"pubno": "...", "pdf_path": "..."}

        # 6) 汇总元数据（保持有序）
        self.meta_schema.update(img_meta)
        self.meta_schema.update(meta_blocks)
        self.meta_schema.update(pubno_info)

        # 7) 组装为 LangChain Document
        doc = Document(page_content=md_text_filtered, metadata=dict(self.meta_schema))
        return doc
        


    def _create_enhanced_metadata(self) -> dict:
        """创建增强的元数据，包含段落-图片映射"""
        enhanced_meta = self.meta_schema.copy()
        
        # 添加段落级图片映射
        enhanced_meta["section_image_map"] = {
            "摘要": self.meta_schema["fig_list"].get("abs_im"),
            "附图说明": self._get_figure_list_paths(),
            # 可以添加更多段落类型
        }
        
        # 添加图片展示配置
        enhanced_meta["image_display_config"] = {
            "摘要图_display_mode": "inline",  # 内联显示
            "附图_display_mode": "gallery",   # 画廊模式
        }
        
        return enhanced_meta

    def _get_figure_list_paths(self) -> List[str]:
        """获取所有附图路径列表"""
        paths = []
        fig_list = self.meta_schema["fig_list"]
        for key, value in fig_list.items():
            if key != "abs_im" and isinstance(value, list) and len(value) == 2:
                paths.append(value[1])  # 路径
        return paths


#   step2： RAG retriever
class PatentImageRetriever:
    """专利图片智能检索器"""
    
    def __init__(self, document: Document):
        self.document = document
        self.metadata = document.metadata
    
    def detect_section_type(self, text_chunk: str) -> str:
        """检测文本片段的段落类型"""
        # 关键词匹配
        if any(keyword in text_chunk for keyword in ["摘要", "本实用新型"]):
            return "摘要"
        elif "附图说明" in text_chunk:
            return "附图说明"
        elif re.search(r"图\d+", text_chunk):
            return "含图段落"
        return "普通段落"
    
    def get_related_images(self, text_chunk: str) -> List[Dict]:
        """根据文本片段获取相关图片"""
        section_type = self.detect_section_type(text_chunk)
        images = []
        
        if section_type == "摘要":
            abs_img = self.metadata["fig_list"].get("abs_im")
            if abs_img:
                images.append({
                    "type": "摘要图",
                    "description": abs_img[0],
                    "path": abs_img[1],
                    "display_mode": "inline"
                })
        
        elif section_type == "含图段落":
            # 提取文本中提到的图号
            fig_nums = re.findall(r"图(\d+)", text_chunk)
            for num in fig_nums:
                fig_key = f"图{num}"
                if fig_key in self.metadata["fig_list"]:
                    fig_info = self.metadata["fig_list"][fig_key]
                    images.append({
                        "type": f"图{num}",
                        "description": fig_info[0],
                        "path": fig_info[1],
                        "display_mode": "reference"
                    })
        
        return images
    
    def format_response_with_images(self, text_chunk: str, images: List[Dict]) -> str:
        """格式化带图片的响应"""
        response = text_chunk
        
        for img in images:
            if img["display_mode"] == "inline":
                response += f"\n\n📸 {img['type']}: {img['description']}\n"
                response += f"[图片路径: {img['path']}]"
            elif img["display_mode"] == "reference":
                response += f"\n\n🔍 参考{img['type']}: {img['description']}"
        
        return response



#   step3
class PatentRAGWithImages:
    """带图片的专利RAG系统"""
    
    def retrieve_and_display(self, query: str) -> str:
        """检索并显示带图片的结果"""
        # 1. 常规RAG检索
        retrieved_docs = self.retriever.retrieve(query)
        
        # 2. 为每个检索结果添加图片
        enhanced_results = []
        for doc in retrieved_docs:
            image_retriever = PatentImageRetriever(doc)
            related_images = image_retriever.get_related_images(doc.page_content)
            
            if related_images:
                enhanced_content = image_retriever.format_response_with_images(
                    doc.page_content, related_images
                )
                enhanced_results.append(enhanced_content)
            else:
                enhanced_results.append(doc.page_content)
        
        return "\n\n---\n\n".join(enhanced_results)
    
    
    
    
    




